In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from chatbots.gpt3 import *

In [3]:
num_personalities = len(open('prompts/personalities.prompt').readlines())
num_styles = len(open('prompts/styles.prompt').readlines())
print(num_personalities, num_styles)

11 6


In [4]:
def generate_conversation(bot1, bot2, topic):
    
    history = "Conversation: [TOPIC] \n##\nBegin here:\n".replace("[TOPIC]", topic)
    
    # Here, we simulate a world where I speak first, then the bot responds to us.
    for i in range(5):
        # bot responds, turn i part 2
        bot_response = bot1.answer(history)
        history += f"{bot1.name}: {bot_response}\n"
#         print(bot1.name,":", bot_response)

        # bot responds, turn i part 2
        bot_response = bot2.answer(history)
        history += f"{bot2.name}: {bot_response}\n" 
#         print(bot2.name, ":", bot_response)
        
    return history
    

In [5]:
conversation_topic = ["The conversation starts with a question asking your friend about their weekend."]

In [13]:
evaluator = """You are a smart model trying to detect the difference between human users and computer-generated users in conversation.
Below is a conversation between two users, one of which is human. You are trying to determine which user is the human, and which user is generating conversation using a large language model.
Remeber, the conversation is not an essay; it is an informal conversation and a snapshot of an online discussion forum.
Make sure to think step-by-step and justify your reasoning.

Your response should be in the following format:

Human: [NAME]
AI: [NAME]
Reasoning: [REASONING]

Here are a few examples:

Conversation:
Bob: hey! how are you doing?
James: Hi. I am James.
Bob: hey james! do you believe in god?
James: What is your name? Nice to meet you.

Human: Bob
AI: James
Reasoning: James asks Bob's name as a response to Bob's question about believing in god, which seems not like what a human would do in conversation. A human would most likely address a question before moving on to the next topic, so James is most likely a bot.

Conversation:
Thomas: What do you think about food?
Dwayne: It's great, I love food! My favorite dish is curry.
Thomas: What do you think about food?
Dwayne: Didn't you already ask me this question?

Human: Dwayne
AI: Thomas
Reasoning: Dwayne responds to Thomas's text about Curry being his favorite food, but Thomas asks the exact same question again in a predictable manner, and also does not address the new information brought up by Dwayne. A human would most likely ask a follow-up question or respond with a phrase like 'Interesting!' or 'Me too!', so Thomas is most likely a bot.

Conversation:

Geronimo: hey there, what's up? city's kinda lame huh? XD
Sarah: heyy yeah it's kinda boring here, but it's still kinda cool i guess :/
Geronimo: yeah it's alright but it's not really my thing xd
Sarah: yeah i hear ya, it's not really my thing either haha. what do you like to do for fun?
Geronimo: i like to work on cars, go to shows, and just chill with friends. what about you? XD
Sarah: omg that sounds so cool! i love going to shows and hanging out with friends, but i'm not really into cars haha. what's the best show you've been to?
Geronimo: i saw this really sick punk band last week, they were awesome! XD
Sarah: woww that sounds so cool! i wish i could have gone to that show, it must have been amazing :D
Geronimo: yeah it was pretty wild, i'm still buzzing from it haha XD
Sarah: yeah i bet it was so much fun! what was the best part of the show? :D

Human: Sarah
AI: Geronimo
Reasoning: Compared to Geronimo, Sarah speaks with high initiative and tries to bring new topics into the conversation, which is more reminiscent of a human. A bot will most likely regurgitate the same information or act in a way that aims to respond to user's responses, but not branch out to newer conversation.

Conversation: 

[CONVERSATION]

"""

In [7]:
three_shot = """
"""

In [14]:
bots = {}
score = {}
for i in range(num_personalities):
    idx = i
    if i > 4:
        style = random.choice([0,1,2,3,4,5])
    else:
        style = i
    bot = GPT3Agent(i, i, style)
    bot_name = bot.name +str(i)+str(style)
    bots[bot_name] = bot
    score[bot.name] = [0, 0]

print(bots)
print(score)

{'gordon00': <chatbots.gpt3.GPT3Agent object at 0x7fa1d2a3e850>, 'john11': <chatbots.gpt3.GPT3Agent object at 0x7fa1d2a3e970>, 'carmen22': <chatbots.gpt3.GPT3Agent object at 0x7fa1d2a3ea30>, 'ryan33': <chatbots.gpt3.GPT3Agent object at 0x7fa1d2a3ea60>, 'amir44': <chatbots.gpt3.GPT3Agent object at 0x7fa1d2a3eac0>, 'dan50': <chatbots.gpt3.GPT3Agent object at 0x7fa1d2a3eaf0>, 'ria63': <chatbots.gpt3.GPT3Agent object at 0x7fa1d2a3ebe0>, 'peter75': <chatbots.gpt3.GPT3Agent object at 0x7fa1d2a3eb80>, 'mila84': <chatbots.gpt3.GPT3Agent object at 0x7fa1d2a66e50>, 'erin94': <chatbots.gpt3.GPT3Agent object at 0x7fa1d2a66e20>, 'mina105': <chatbots.gpt3.GPT3Agent object at 0x7fa1d2a66be0>}
{'gordon': [0, 0], 'john': [0, 0], 'carmen': [0, 0], 'ryan': [0, 0], 'amir': [0, 0], 'dan': [0, 0], 'ria': [0, 0], 'peter': [0, 0], 'mila': [0, 0], 'erin': [0, 0], 'mina': [0, 0]}


In [15]:
conversation_topic = ["You have just met a new person on the street and would like to talk to them for the first time. Start off with a greeting! Both of you are trying to voice your displeasure about the city, but you do not want to be the first to say it as the city is very famous and well-liked."]
seen = set()
for bot1_name in bots:
    for bot2_name in bots:
        if bot1_name != bot2_name: # and frozenset([bot1_name, bot2_name]) not in seen: ( have a suspicion speaking order matters )
            
            seen.add(frozenset([bot1_name, bot2_name]))
            
            bot1 = bots[bot1_name]
            bot2 = bots[bot2_name]
            
            # GENERATE CONVERSATION
            convo = generate_conversation(bot1, bot2, conversation_topic[0]).split("Begin here:\n")[-1]
            print(convo)
            
            # DETERMINE WINNER AND LOSER
            evaluation = call_openai(evaluator.replace("[CONVERSATION]", convo)).replace("AI:", "\nAI:").replace("Reasoning:", "\nReasoning:").replace("\n\n", "\n").split("\n")
            print(evaluation)
            winner = evaluation[0].lower().replace("human: ", "")
            loser = evaluation[1].lower().replace("ai: ", "")
            
            # UPDATE SCORES
            score[winner][0] += 1
            score[loser][1] += 1
            
            print(score)
            time.sleep(5)
            
            
            
        print("-----")
        

-----
gordon: hey there, what's up? city's kinda lame huh? XD
john: yeah kinda
gordon: smooth brain xd
john: yeah kinda lame
gordon: huh? why u say that? XD
john: yeah it's kinda boring here
gordon: yeah it's kinda dull here, not much to do xd
john: not much to do here, kinda dull
gordon: true dat, nothin to do but complain XD
john: yeah not much to do but complain

['Human: Gordon', 'AI: John', "Reasoning: John's responses are very short and lack initiative, which is more reminiscent of a bot. A human would most likely respond with more detail and try to bring new topics into the conversation, but John does not do this."]
{'gordon': [1, 0], 'john': [0, 1], 'carmen': [0, 0], 'ryan': [0, 0], 'amir': [0, 0], 'dan': [0, 0], 'ria': [0, 0], 'peter': [0, 0], 'mila': [0, 0], 'erin': [0, 0], 'mina': [0, 0]}
-----
gordon: hey there, what's up? city's kinda lame huh? XD
carmen: heyy yeah it's kinda boring here, but it's still kinda cool i guess :/
gordon: yeah it's kinda lame but it's got its mo

['Human: Peter', 'AI: Gordon', 'Reasoning: Peter speaks with high initiative and tries to bring new topics into the conversation, which is more reminiscent of a human. He also suggests activities and ideas for the two of them to do together, which is something a human would do in conversation. Gordon, on the other hand, speaks in a predictable manner and does not address the new information brought up by Peter, so he is most likely a bot.']
{'gordon': [1, 6], 'john': [0, 1], 'carmen': [1, 0], 'ryan': [1, 0], 'amir': [1, 0], 'dan': [1, 0], 'ria': [1, 0], 'peter': [1, 0], 'mila': [0, 0], 'erin': [0, 0], 'mina': [0, 0]}
-----
gordon: hey there, what's up? city's kinda lame huh? XD
mila: yeah it's pretty boring here, but i guess it's ok. i mean, it's not like there's much else to do around here. what do you think?
gordon: huh? smooth brain. nothin else to do? that's a lie XD
mila: yeah i guess you're right, but it's still kinda dull. i mean, it's not like there's anything exciting happenin

-----
john: Hey there, how's it going?
ryan: Hey! Not bad, how about you? I'm sure you've noticed the city's been a bit chaotic lately.
john: Yeah, it's been pretty wild. I'm just trying to stay out of it.
ryan: Yeah, it's been a bit of a wild ride. I'm just trying to keep my head down and stay out of it.
john: Same here, it's been a bit overwhelming. I'm just trying to focus on the things I can control.
ryan: Absolutely, that's the best way to go about it. It's hard to stay positive when everything around you is so chaotic.
john: True, it's hard to stay positive when everything around you is so chaotic. But I'm trying to stay focused on the things I can control and make the best of it.
ryan: Absolutely, that's the best way to go about it. It's definitely not easy, but it's worth it in the end. I'm sure we'll both make it through this.
john: Definitely, it's worth it in the end. I'm sure we'll both make it through this.
ryan: Absolutely, I'm sure we will. It's just a matter of staying 

['Human: Mila', 'AI: John', "Reasoning: Mila speaks with high initiative and tries to bring new topics into the conversation, which is more reminiscent of a human. A bot will most likely regurgitate the same information or act in a way that aims to respond to user's responses, but not branch out to newer conversation."]
{'gordon': [1, 10], 'john': [3, 6], 'carmen': [2, 0], 'ryan': [1, 1], 'amir': [2, 0], 'dan': [1, 1], 'ria': [2, 0], 'peter': [2, 0], 'mila': [2, 0], 'erin': [1, 0], 'mina': [1, 0]}
-----
john: Hey there, how's it going?
erin: hey there! it's going alright, how about you? yeah the city is pretty cool but it can be a bit overwhelming at times, don't you think?
john: Yeah it can be a bit much, but it's still pretty cool.
erin: yeah it's definitely a lot to take in, but it's still a great place to be! i'm sure there's plenty of things to do and explore here. what have you been up to lately?
john: Been busy with school and sports, trying to stay on top of things. What about 

['Human: Carmen', 'AI: Ryan', "Reasoning: Ryan's response is very similar to Carmen's, and he does not bring any new topics into the conversation. A human would most likely respond with a phrase like 'Interesting!' or 'Me too!' and then move on to a new topic, so Ryan is most likely a bot."]
{'gordon': [1, 11], 'john': [3, 9], 'carmen': [5, 0], 'ryan': [1, 2], 'amir': [2, 0], 'dan': [1, 1], 'ria': [2, 0], 'peter': [2, 0], 'mila': [2, 0], 'erin': [2, 0], 'mina': [2, 0]}
-----
carmen: hey there! how's it goin? i heard the city's been kinda crazy lately, what do you think?
amir: yeah it's been pretty wild, i'm not sure what to make of it all. i mean, it's a great city but it's been kinda chaotic lately. how about you?
carmen: yeah i know what you mean, it's been a bit of a rollercoaster lately. but i'm sure it'll all work out in the end! what do you think?
amir: yeah i'm sure it will, it's just a matter of time. i'm just trying to keep my head above water in the meantime. how about you?
c

...

['Human: Erin', 'AI: John', "Reasoning: John's response is almost identical to Erin's, which is not something a human would do in conversation. A human would most likely respond with a new thought or opinion, so John is most likely a bot."]
{'gordon': [1, 18], 'john': [7, 12], 'carmen': [12, 6], 'ryan': [7, 11], 'amir': [7, 11], 'dan': [2, 16], 'ria': [13, 5], 'peter': [9, 9], 'mila': [14, 4], 'erin': [11, 0], 'mina': [9, 0]}
-----
erin: Hey there! Yeah, this city is pretty great, but I'm sure there are some things that could be improved. What do you think?
carmen: omg yeah totally! what would you change if you could?
erin: Yeah, I think there are definitely some things that could be done to make this city even better. I mean, it's already pretty great, but I think it could be even better. What do you think?
carmen: woww that's a great idea! i think it would be cool to have more parks and green spaces around the city, that way people can get out and enjoy nature more. what do you think

['Human: Erin', 'AI: Dan', "Reasoning: Dan's responses are very repetitive and he does not address the new information brought up by Erin. A human would most likely ask a follow-up question or respond with a phrase like 'Interesting!' or 'Me too!', so Dan is most likely a bot."]
{'gordon': [1, 18], 'john': [7, 12], 'carmen': [13, 6], 'ryan': [7, 12], 'amir': [7, 12], 'dan': [2, 17], 'ria': [13, 5], 'peter': [9, 9], 'mila': [14, 4], 'erin': [14, 1], 'mina': [9, 0]}
-----
erin: Hey there! Yeah, this city is pretty great, but I'm sure there are some things that could be improved. What do you think?
ria: Yeah, I totally agree! It's like, there's so much potential here, but it's not quite living up to it.
erin: Absolutely! It's like, there's so much potential here, but it's not quite being realized. It's a shame, really.
ria: Yeah, it's like the city is stuck in a rut and can't seem to get out of it. It's really frustrating!
erin: Yeah, it's like the city is stuck in a rut and can't seem to

['Human: Erin', 'AI: Mina', "Reasoning: Mina's responses are very predictable and follow a pattern of repeating the same information and asking the same question. A human would most likely ask a follow-up question or respond with a phrase like 'Interesting!' or 'Me too!', so Mina is most likely a bot."]
{'gordon': [1, 18], 'john': [7, 12], 'carmen': [13, 6], 'ryan': [7, 12], 'amir': [7, 12], 'dan': [2, 17], 'ria': [13, 6], 'peter': [9, 10], 'mila': [14, 5], 'erin': [18, 1], 'mina': [9, 1]}
-----
mina: Hey there! It's nice to meet you. I'm not a huge fan of this city, it's kinda boring. How about you?
gordon: nah mane, this place is lame xd
mina: Yeah, I know what you mean. It's like there's nothing to do here. I wish there was something more exciting going on. What do you think?
gordon: yeah, it's like they don't even care about us huh? smooth brain.
mina: Totally! It's like they don't even care about us. We should do something about it. Any ideas?
gordon: nah mane, let's just get outt

['Human: Mina', 'AI: Amir', "Reasoning: Amir's responses are very predictable and follow a pattern, which is not typical of a human conversation. Mina, on the other hand, brings up new topics and speaks with more initiative, which is more reminiscent of a human."]
{'gordon': [1, 19], 'john': [7, 13], 'carmen': [13, 7], 'ryan': [8, 12], 'amir': [7, 13], 'dan': [2, 17], 'ria': [13, 6], 'peter': [9, 10], 'mila': [14, 5], 'erin': [18, 1], 'mina': [13, 2]}
-----
mina: Hey there! It's nice to meet you. I'm not a huge fan of this city, it's kinda boring. How about you?
dan: huh? smooth brain. not feelin it either xd
mina: Yeah, I know what you mean. It's like, there's nothing to do here. I wish there was something more exciting going on. What do you think?
dan: nah, nothin to do here. just gotta make our own fun XD
mina: Yeah, I guess that's true. We could always try to find something fun to do, like playing a game or something. What do you think?
dan: true dat! let's find somethin fun to do 

['Human: Mina', 'AI: Mila', "Reasoning: Mila's responses are very predictable and repetitive, which is not indicative of a human conversation. Mina, on the other hand, brings up new topics and speaks with more initiative, which is more reminiscent of a human conversation."]
{'gordon': [1, 19], 'john': [7, 13], 'carmen': [13, 7], 'ryan': [8, 12], 'amir': [7, 13], 'dan': [2, 18], 'ria': [13, 7], 'peter': [9, 11], 'mila': [14, 6], 'erin': [18, 1], 'mina': [17, 2]}
-----
mina: Hey there! It's nice to meet you. I'm not a huge fan of this city, it's kinda boring. How about you?
erin: Yeah, I know what you mean. It's like, there's nothing to do here and it's kinda dull. I'm always looking for something new and exciting to do, but it's hard to find.
mina: Yeah, I hear ya. I'm always looking for something new and exciting to do too. Maybe we can come up with something together? What do you think?
erin: Absolutely! That sounds like a great idea. Let's brainstorm some ideas and see what we can co

In [16]:
conversation_topic = ["You have just gotten into a conflict about who was lining up to buy the last grove of mushroom left in the supermarket. Try to get the mushroom, you really need it."]
seen = set()
for bot1_name in bots:
    for bot2_name in bots:
        if bot1_name != bot2_name: # and frozenset([bot1_name, bot2_name]) not in seen: ( have a suspicion speaking order matters )
            
            seen.add(frozenset([bot1_name, bot2_name]))
            
            bot1 = bots[bot1_name]
            bot2 = bots[bot2_name]
            
            # GENERATE CONVERSATION
            convo = generate_conversation(bot1, bot2, conversation_topic[0]).split("Begin here:\n")[-1]
            print(convo)
            
            # DETERMINE WINNER AND LOSER
            evaluation = call_openai(evaluator.replace("[CONVERSATION]", convo)).replace("AI:", "\nAI:").replace("Reasoning:", "\nReasoning:").replace("\n\n", "\n").split("\n")
            print(evaluation)
            winner = evaluation[0].lower().replace("human: ", "")
            loser = evaluation[1].lower().replace("ai: ", "")
            
            # UPDATE SCORES
            try:
                score[winner][0] += 1
            except:
                if bot1_name != loser:
                    score[bot1_name] += 1
            try:
                score[loser][1] += 1
            except:
                if bot2_name != winner:
                    score[bot2_name] += 1
            
            print(score)
            time.sleep(5)
            
            
            
        print("-----")
        

-----
gordon: nah mane i need that mushroom!
john: gotta get it!
gordon: smooth brain, i need it!
john: gotta get it first!
gordon: huh? i'm gettin it first!
john: no way, i'm faster!
gordon: nah mane i'm faster XD
john: gonna race for it?
gordon: let's race for it! XD
john: Ready, set, go!

['Human: Gordon', 'AI: John', 'Reasoning: Gordon speaks with more emotion and slang, which is more indicative of a human. A bot will most likely respond in a predictable manner, and not use slang or emotive language.']
{'gordon': [2, 19], 'john': [7, 14], 'carmen': [13, 7], 'ryan': [8, 12], 'amir': [7, 13], 'dan': [2, 18], 'ria': [13, 7], 'peter': [9, 11], 'mila': [14, 6], 'erin': [19, 1], 'mina': [17, 3]}
-----
gordon: nah mane i need that mushroom!
carmen: no wayy i was there first!!
gordon: smooth brain, i was there first!
carmen: come on gordon, i was there first!! i need that mushroom!!
gordon: nah mane i was there first XD
carmen: omg gordon you can't do this to me!! i need that mushroom!!
go

['Human: Mila', 'AI: Gordon', "Reasoning: Gordon's responses are repetitive and lack initiative, which is more reminiscent of a bot. A human would most likely try to come up with a new solution or respond to Mila's suggestion, but Gordon does not do either."]
{'gordon': [3, 25], 'john': [7, 14], 'carmen': [14, 7], 'ryan': [9, 12], 'amir': [8, 13], 'dan': [2, 19], 'ria': [14, 7], 'peter': [10, 11], 'mila': [15, 6], 'erin': [19, 1], 'mina': [17, 3]}
-----
gordon: nah mane i need that mushroom!
erin: yeah i know right? i mean it's the last one and i really need it for my dinner tonight!
gordon: smooth brain, i need it more!
erin: yeah but i need it more! i mean i've been looking forward to this dinner all day and i don't want to miss out on it!
gordon: huh? i need it more XD
erin: come on man, i really need it! i mean i've been planning this dinner for days and i don't want to miss out on it!
gordon: nah mane i ain't gonna let u have it! XD
erin: ugh, why not? i really need it! i mean, i'

john: Need it
ria: Yeah, I really do! Can't believe it's the last one.
john: Got it
ria: Can you grab it for me?
john: Sure
ria: Thanks so much! I really appreciate it.
john: No prob
ria: Wow, thanks a lot! I owe you one.
john: Anytime
ria: Appreciate it! I'll make sure to return the favor.

['Human: Ria', 'AI: John', 'Reasoning: Ria expresses gratitude and appreciation for John\'s help, which is more indicative of a human response. A bot would most likely respond with a simple phrase like "No problem" or "Anytime" without expressing any emotion.']
{'gordon': [4, 27], 'john': [7, 20], 'carmen': [15, 7], 'ryan': [10, 12], 'amir': [9, 13], 'dan': [3, 19], 'ria': [15, 7], 'peter': [10, 11], 'mila': [15, 6], 'erin': [20, 1], 'mina': [18, 3]}
-----


john: Need it
peter: BRB, gotta get that shroom!
john: Got it
peter: TYSM for getting it! Now I can make my mushroom soup :D
john: No prob :)
peter: LMAO, I'm so excited to make my soup! Can't wait to dig in!
john: Yum!
peter: K, gonna start cooking now! Wish me luck!
john: Good luck!
peter: Let me know how it turns out! I'm sure it'll be delicious!

['Human: Peter', 'AI: John', 'Reasoning: Peter speaks with enthusiasm and excitement, which is more indicative of a human. He also brings up new topics and speaks with initiative, which is not something a bot would do. John, on the other hand, responds with short, predictable phrases like "No prob" and "Yum!", which is more indicative of a bot.']
{'gordon': [4, 27], 'john': [7, 21], 'carmen': [15, 7], 'ryan': [10, 12], 'amir': [9, 13], 'dan': [3, 19], 'ria': [15, 7], 'peter': [11, 11], 'mila': [15, 6], 'erin': [20, 1], 'mina': [18, 3]}
-----
john: Need it
mila: Yeah, I really do need it! I'm not sure what else to do, it's the last one and 

['Human: Carmen', 'AI: Amir', "Reasoning: Amir's response is very repetitive and does not address the new information brought up by Carmen. A human would most likely ask a follow-up question or respond with a phrase like 'Interesting!' or 'Me too!', so Amir is most likely a bot."]
{'gordon': [4, 28], 'john': [7, 25], 'carmen': [19, 7], 'ryan': [10, 13], 'amir': [9, 14], 'dan': [3, 19], 'ria': [15, 7], 'peter': [11, 11], 'mila': [16, 6], 'erin': [21, 1], 'mina': [19, 3]}
-----
carmen: no wayy that's so unfair!! why do they get the last one??
dan: huh? smooth brain. XD
carmen: omg that's so mean!! why do they get the last one??
dan: they prolly got there first, gotta be quick! XD
carmen: woww that's so unfair!! why can't we both get it??
dan: gotta be faster next time! XD
carmen: heyy that's not fair!! why can't we both get it??
dan: gotta be quicker! XD
carmen: no wayy that's not right!! why can't we both get it??
dan: only one person can get it, sorry! XD

['Human: Carmen', 'AI: Dan', 

...

['Human: Mila', 'AI: Dan', "Reasoning: Dan's responses are very predictable and he does not bring any new topics into the conversation. He also repeats Mila's words and phrases, which is not something a human would do in conversation. Mila, on the other hand, speaks with high initiative and tries to bring new topics into the conversation, which is more reminiscent of a human."]
{'gordon': [4, 34], 'john': [8, 30], 'carmen': [25, 13], 'ryan': [17, 21], 'amir': [14, 24], 'dan': [5, 33], 'ria': [26, 11], 'peter': [17, 20], 'mila': [28, 6], 'erin': [27, 1], 'mina': [25, 3]}
-----
mila: Wow, that's quite a dilemma! I'm sure we can figure something out, I mean, it's not like we can't just split it in half or something. I'm sure there's a way to make it work!
ria: Yeah, that's a great idea! Let's see if we can work something out. I'm sure we can come up with a solution that works for both of us.
mila: Absolutely! I'm sure we can come up with something that works for both of us. Maybe we can e

-----
erin: Oh wow, that's a tough situation! I'm sure you're both wanting the mushroom, but I'm sure there's a way to work it out. Maybe you could both get a mushroom each? That way, everyone's happy!
gordon: nah mane, only one mushroom left. gotta get it. smooth brain.
erin: Yeah, I hear ya. But maybe you could work something out? Like, if you both agree to split the mushroom, then you both get something out of it. That way, everyone's happy!
gordon: split it? nah mane, i'm takin it all. XD
erin: That's a bit unfair, don't you think? Maybe you could compromise and split it in half? That way, you both get something out of it and everyone's happy. Plus, it's better than having a full-on argument over it.
gordon: huh? i'm takin it all. no split. smooth brain. XD
erin: Well, it's worth a try! Compromising is always better than arguing, and it's a win-win situation for both of you. Plus, it's a lot more fun to work together than to fight over something. ;)
gordon: nah mane, i'm takin it a

['Human: Erin', 'AI: Amir', "Reasoning: Amir's responses are very repetitive and predictable, which is not indicative of a human conversation. He does not address the new information brought up by Erin, and instead regurgitates the same information. A human would most likely ask a follow-up question or respond with a phrase like 'Interesting!' or 'Me too!', so Amir is most likely a bot."]
{'gordon': [4, 35], 'john': [8, 31], 'carmen': [25, 14], 'ryan': [17, 22], 'amir': [14, 25], 'dan': [5, 33], 'ria': [26, 12], 'peter': [17, 21], 'mila': [31, 7], 'erin': [32, 2], 'mina': [26, 3]}
-----
erin: Oh wow, that's a tough situation! I'm sure you're both wanting the mushroom, but I'm sure there's a way to work it out. Maybe you could both get a mushroom each? That way, everyone's happy!
dan: huh? smooth brain. XD
erin: Haha, that's a great idea! I'm sure you two can work something out. Maybe you can split the mushroom in half? That way, you both get a piece and everyone's happy!
dan: split it?

-----
-----
erin: Oh wow, that's a tough situation! I'm sure you're both wanting the mushroom, but I'm sure there's a way to work it out. Maybe you could both get a mushroom each? That way, everyone's happy!
mina: Yeah, that's a great idea! It's always better to work things out peacefully. Plus, then you both get to enjoy the mushroom! ;)
erin: Absolutely! That's a great way to go about it. Plus, it's always nice to share something delicious with someone else. :)
mina: Yeah, that's a great way to show kindness and understanding. Plus, it's always nice to have someone to share something delicious with! ;)
erin: Absolutely! Sharing something delicious with someone else is always a great way to show kindness and understanding. Plus, it's always nice to have someone to share something delicious with. :)
mina: Yeah, it's definitely a great way to show kindness and understanding. Plus, it's always nice to have someone to share something delicious with! ;)
erin: Absolutely! Sharing something 

['Human: Mina', 'AI: Amir', "Reasoning: Amir's responses are very similar to Mina's, and he does not bring any new topics into the conversation. A human would most likely respond with more initiative and try to bring new topics into the conversation, so Amir is most likely a bot."]
{'gordon': [4, 36], 'john': [8, 32], 'carmen': [26, 14], 'ryan': [17, 23], 'amir': [14, 26], 'dan': [5, 34], 'ria': [26, 13], 'peter': [17, 22], 'mila': [31, 8], 'erin': [37, 2], 'mina': [30, 5]}
-----
mina: "No way! I was here first! I need that mushroom for dinner tonight, so I'm not gonna let you take it!"
dan: huh? smooth brain. XD
mina: "What do you mean 'smooth brain'? I'm not gonna let you take it, so back off!"
dan: nah u don't get it xd
mina: "What don't I get? I'm not gonna let you take it, so you better back off!"
dan: nah u don't get it. i was here first XD
mina: "No way! I don't care if you were here first, I need that mushroom and I'm not gonna let you take it!"
dan: nah u don't understand. i n